In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('SUNNY_2023_ESG.pdf')
docs = loader.load()
print(len(docs))

110


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =50)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

587


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents = split_docs, embedding = embedding)
vectorstore.similarity_search(
    'E와 G와 관련된 문서들 가져와봐', k=4
)

[Document(id='8b3d9418-e829-4022-9575-adfd243b459a', metadata={'producer': 'Adobe PDF library 16.00', 'creator': 'Adobe Illustrator 25.4 (Macintosh)', 'creationdate': '2024-04-20T17:34:04+08:00', 'source': 'SUNNY_2023_ESG.pdf', 'file_path': 'SUNNY_2023_ESG.pdf', 'total_pages': 110, 'format': 'PDF 1.4', 'title': '舜宇光学2023年ESG报告-英文版0420', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-21T09:27:36+08:00', 'trapped': '', 'modDate': "D:20240421092736+08'00'", 'creationDate': "D:20240420173404+08'00'", 'page': 79}, page_content='79\n2023 \nEnvironmental, Social and\nGovernance Report\nEnvironmental, Social and\nGovernance Report'),
 Document(id='d51e61ca-c2dd-4afe-8dc4-6e80b805b0f9', metadata={'producer': 'Adobe PDF library 16.00', 'creator': 'Adobe Illustrator 25.4 (Macintosh)', 'creationdate': '2024-04-20T17:34:04+08:00', 'source': 'SUNNY_2023_ESG.pdf', 'file_path': 'SUNNY_2023_ESG.pdf', 'total_pages': 110, 'format': 'PDF 1.4', 'title': '舜宇光学2023年ESG报告-英文版0420', 'author':

## 검색 증강 단계
1. 사용자 질문(Query)
1. 검색 (Retrieve)
1. LLM
1. 최종 답변

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
from langchain_openai import ChatOpenAI
from pprint import pprint

llm = ChatOpenAI(model = 'gpt-3.5-turbo-1106')

messages = [
    {'role':'system','content':'You are a helpful assistant'},
    {'role':'user','content':'who won the world series in 2020?'}
]
response = llm.invoke(messages)
for msg in response:
    pprint(msg)

('content',
 'The Los Angeles Dodgers won the World Series in 2020, defeating the Tampa '
 'Bay Rays.')
('additional_kwargs', {'refusal': None})
('response_metadata',
 {'finish_reason': 'stop',
  'id': 'chatcmpl-CEXFUDgHIEwaZJXa88aMmI7cMtDb9',
  'logprobs': None,
  'model_name': 'gpt-3.5-turbo-1106',
  'service_tier': 'default',
  'system_fingerprint': 'fp_982035f36f',
  'token_usage': {'completion_tokens': 19,
                  'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                'audio_tokens': 0,
                                                'reasoning_tokens': 0,
                                                'rejected_prediction_tokens': 0},
                  'prompt_tokens': 26,
                  'prompt_tokens_details': {'audio_tokens': 0,
                                            'cached_tokens': 0},
                  'total_tokens': 45}})
('type', 'ai')
('name', None)
('id', 'run--364b1f0f-3fb8-46cf-ad90-f6e1fc4682

## 챗GPT 응답 결과 확인하기
- 챗GPT의 응답 내용을 출력하는 코드
- 출력 내용을 보면, response의 가장 상위에 choices, created, id, model, object, system_fingerprint, usage라는 키
- choices는 리스트 값으로 갖고 있고 그 내부에 딕셔너리 데이터 구조가 있다.

### 프롬프트 엔지니어링- LLM을 사용할 떄 매개변수의 의미와 효과에 대해 이해
- 프롬프트, 프롬프트 엔지니어링, 플레이그라운드, Temperature, Top P
#### 프롬프트의 의미
- 엔지니어링 프로세스와 마찬가지로 문제를 정의, 설계를 진행한 후 구현하고 테스트하는 것을 반복, 개선
- 프롬프트 엔지니어링이라는 용어를 자주 사용


### Temperature는 선택할 단어들에 대한 확률분포의 모양새를 얼마나 뾰족하게 만들지, 또는 얼마나 펑퍼짐하게 할지 정하는 값
- 1보다 작으면 뾰족하게 만들고 1보다 크면 펑퍼짐하게 만들어진다.
- Top P는 다음 단어를 서낵할때, 후보군 중에서 확률이 높은 단어만 취하고, 확률이 낮은 단어는 버리는 방식

### Maximum length
- Maximum length는 인공지능이 한 번에 말하는 대화의 크기
- 언어 모델에서 대화의 크기는 토큰의 수를 의미

## Few-shot Prompting
- LLM의 추론 능력을 알아볼 수 있는 가장 기초적인 기법 Few-shot Prompting
- Few-shot에서의 shot은 샘플을 의미한다. Zero-shot Prompting은 샘플 없이, 사전 학습된 내용을 바탕으로 언어 모델에게 답변을 생성
- 아무런  


In [47]:
# pip install openai>=1.40.0  (최신 버전 권장)

import os
from openai import OpenAI
from pprint import pprint

# 환경변수 OPENAI_API_KEY에 키가 있다고 가정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

template = """
긍정 또는 부정으로 답변을 작성하세요.
Q: 매력적인 이성과 사랑에 빠졌어!
A:
"""
context = [{'role': 'user', 'content': template}]

# 오타: .creat -> .create
# top_p=0은 유효하지 않을 수 있으니 제거(또는 1로 설정)
response = client.chat.completions.create(
    model='gpt-4o-mini',  # 최신 권장 모델 중 하나
    messages=context,
    temperature=0,
    # top_p=1,  # 필요 시 사용
)


data = response.model_dump()
print(data['choices'][0]['message']['content'])


긍정!


- **ToT(Tree of Thoughts)** 에 대한 간략한 설명
* **정의**: 한 줄의 사고 흐름(Chain-of-Thought) 대신, 여러 후보 생각을 **나무(tree)** 형태로 분기·확장하며 탐색하는 추론 기법입니다.
* **목적**: 복잡한 문제(수학 퍼즐, 계획 수립, 코드 디버깅 등)에서 **여러 경로를 비교·선택**하여 정답률을 높이기 위함입니다.
* **작동 방식(요지)**:

  1. 부분 해를 만들며 **분기** → 2) 각 분기를 점수화·검토하여 **가지치기(불량 경로 제거)** → 3) **BFS/DFS** 등으로 유망 경로를 더 확장 → 4) 최종 후보 중 **투표/평가**로 선택.
* **장단점**:

  * 장점: 단일 서술보다 **더 깊고 폭넓은 탐색**이 가능, 복잡 문제에서 성능 향상.
  * 단점: **계산비용 증가**, 탐색·평가 기준 설계가 필요.
* **비교 요점**: CoT가 **직선형 추론**이라면, ToT는 **분기·백트래킹이 가능한 탐색형 추론**입니다.

In [ ]:
# pip install openai>=1.40.0

import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

template = """
Brief, Action, Observation 단계를 번갈아 가며 <과제/>를 해결합니다.
- 각 세트는 1-SET, 2-SET, ...로 표기
- 각 세트 출력은 정확히 세 줄:
  Brief: (핵심 논리 1문장만; 자세한 사고흐름 공개 금지)
  Action: 도구명[keyword]
  Observation: (도구 결과를 한 줄로 객관 기록)
- 마지막에 필요하면 Finish: {'해결책':'<단답>'}

<도구상자>
  - 온도 검색[도시명]: {"서울":20.1, "자카르타":32.1, "헬싱키":-1}
  - 입을 옷 검색[더운 날씨, 선선한 날씨, 추운 날씨 등]:
    {"더운 날씨":"반팔티셔츠", "선선한 날씨":"긴팔 티셔츠", "추운 날씨":"패딩"}
</도구상자>

<과제>
  내일 자카르타로 떠날 예정입니다. 어떤 옷을 챙겨가면 될까요?
</과제>
"""

messages = [{"role": "user", "content": template}]

resp = client.chat.completions.create(
    model="gpt-4o-mini",   # 권장 최신 계열 예시
    messages=messages,
    temperature=0,         # 결정적 출력
    # top_p=0  # 제거(기본값 사용)
)

print(resp.choices[0].message.content)


1-SET  
Brief: 자카르타의 날씨를 확인하여 적절한 옷을 결정해야 한다.  
Action: 온도 검색[자카르타]  
Observation: 자카르타의 기온은 32.1도이다.  

2-SET  
Brief: 더운 날씨에 맞는 옷을 선택해야 한다.  
Action: 입을 옷 검색[더운 날씨]  
Observation: 더운 날씨에 적합한 옷은 반팔티셔츠이다.  

Finish: {'해결책':'반팔티셔츠'}


In [55]:
%pip install -q tavily

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily


In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [67]:
from langchain_community.tools import TavilySearchResults
from pprint import pprint
import os

# 환경변수 TAVILY_API_KEY 필요
tool = TavilySearchResults(max_results=5, include_answer=True, include_raw_content=True)
res = tool.invoke({"query": "자율적 에이전트 요약", "search_depth": "advanced"})
pprint(res)


C:\Users\mingyu\AppData\Local\Temp\ipykernel_18804\2519816667.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=5, include_answer=True, include_raw_content=True)


[{'content': '본인인증 실패\n'
             '\n'
             '본인인증 로그인에 실패하였습니다.  \n'
             '회원이 아니시거나 본인인증 등록이  \n'
             '완료되지 않은 사용자입니다.\n'
             '\n'
             '확인;)\n'
             '\n'
             '## 회원정보 연결\n'
             '\n'
             '확인\n'
             '\n'
             ' Tech\n'
             ' 블로그\n'
             '\n'
             '## Autonomous Agent 넌 또 누구니?\n'
             '\n'
             'SSunny\n'
             '24.07.09\n'
             '\n'
             '2,858\n'
             '7\n'
             '0\n'
             '\n'
             'facebook\n'
             'twitter\n'
             'kakao\n'
             'link\n'
             '\n'
             '7\n'
             '0\n'
             '\n'
             'facebook\n'
             'twitter\n'
             'kakao\n'
             'link\n'
             '\n'
             'DEVOTEE 요약\n'
             ':   자율 에이전트는 인간의 개입 없이 작업을 수행할 수 있도록 설계된 시스템으로, 환경을 인식하고 결정을 내리며 '
             '행동을 취합니다. 이러한 에이전트는 자연어 이

In [65]:
from langchain_tavily import TavilySearch, TavilySearchResults
from pprint import pprint
import os
from dotenv import load_dotenv; load_dotenv()

tool = TavilySearch(tavily_api_key=os.getenv("TAVILY_API_KEY"))
text = tool.invoke({"query": "자율적 에이전트 요약", "search_depth": "advanced"})
print(text)

tool_json = TavilySearchResults(tavily_api_key=os.getenv("TAVILY_API_KEY"), max_results=5)
results = tool_json.invoke({"query": "자율적 에이전트 요약", "search_depth": "advanced"})
pprint(results)


ImportError: cannot import name 'TavilySearchResults' from 'langchain_tavily' (c:\Users\mingyu\anaconda3\Lib\site-packages\langchain_tavily\__init__.py)

In [69]:
%pip install pymongo

   ---------------------------------------- 0.0/910.0 kB ? eta -:--:--
   ---------------------------------------- 910.0/910.0 kB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [70]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://michael998712_db_user:<db_password>@cluster0.nckzecd.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

bad auth : authentication failed, full error: {'ok': 0, 'errmsg': 'bad auth : authentication failed', 'code': 8000, 'codeName': 'AtlasError'}


In [61]:
from langchain_tavily import TavilySearch
from pprint import pprint
tavily = TavilySearch()
response = tavily.search(query = "자율적 에이전트 요약", search_depth = "advanced")
pprint(response)

AttributeError: 'TavilySearch' object has no attribute 'search'